<a href="https://colab.research.google.com/github/aashitadutta/FinancialNarrativeSummarization/blob/main/TextRank_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import os

In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
inp_data="/content/drive/MyDrive/lstm-input/"
out_data="/content/drive/MyDrive/lstm-summ/"

datasets={"text":inp_data,"summ":out_data}

data_categories=["training","validation","test"]

data={"articles":[],"summaries":[]}

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
!pip install Rouge

In [ ]:
!pip install rouge_score

In [ ]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-06-28 10:11:57--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.224.187
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.224.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  39.3MB/s    in 39s     

2021-06-28 10:12:37 (39.8 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

gzip: GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
import math

In [ ]:
def parsetext(dire,category,filename):
    with open("%s/%s"%(dire,filename),'r',encoding="Latin-1") as readin:
        print("file read successfully")
        text=readin.read()
    return text.lower()


def load_data(dire,category):
    for (root,dirs,files) in os.walk(dire):
         filenames=sorted(files)
    return filenames


def cleantext(text):
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

def printArticlesum(k):
    print("---------------------original sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["articles"][k])
    print("----------------------Summary sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["summaries"][k])
    return 0


def announcedone():
    duration=2000
    freq=440
    ws.Beep(freq,duration)

dict1={}
f1_score1=[]
precision1=[]
recall1=[]
f1_score2=[]
precision2=[]
recall2=[]
f1_scoreL=[]
precisionL=[]
recallL=[]
filenames1=load_data(datasets["text"],data_categories[0])
filenames2=load_data(datasets["summ"],data_categories[0])
print(filenames1)
print(filenames2)

"""----------load the data, sentences and summaries-----------"""
data["articles"]=[]
data["summaries"]=[]
k=0
while k<10:
 try:
    print(filenames1[k])
    data["articles"].append(parsetext(datasets["text"],data_categories[0],"%s"%filenames1[k]))
 except Exception as e:
    data["articles"].append("Could not read")
    print(e)
 try:
   print(filenames2[k])
   data["summaries"].append(parsetext(datasets["summ"],data_categories[0],"%s"%filenames2[k]))
 except Exception as e:
   data["summaries"].append("Could not read")
   print(e)
 
 reference=""
 reference=data["summaries"][k]
 print("number of words in reference summary",len(reference))
 s = ""
 d = {}
 for a in data['articles'][k]:
      s += a
 #print("....")
 print("number of words in text file",len(s))
 # print s
 sentences = sent_tokenize(s)
 print("number of sentences in text file",len(sentences))
 clean_sentences = []
 for s in sentences:
    temp = re.sub("[^a-zA-Z]"," ",s)
    temp = temp.lower()
    clean_sentences.append(temp)
    d[temp] = s 
 #print(len(clean_sentences))
 stop_words = stopwords.words('english')
 def rem_stop(s):
    var = ""
    words = nltk.word_tokenize(s)
    for w in words:
        if( w not in stop_words):
           var+=w+" "
    return var
 dict = {}
 clean = []
 # print clean_sentences
 for s in clean_sentences:
    temp = rem_stop(s)
    clean.append(temp)
    dict[temp] = d[s]
 #print(len(clean))
 ### creating vector representation of sentences after extracting word vectors
 
 # print(model)
 word_embeddings = {}
 words = list(model.wv.vocab)
 # print len(words)
 for a in words:
    word_embeddings[a]=model[a]
 
 # print len(word_embeddings)
 
 
 sentence_vectors = []
 for i in clean:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((300,))
  sentence_vectors.append(v)
 #print(len(sentence_vectors))
 #print(len(sentence_vectors[1]))
 
 sentence_similarity_martix = np.zeros([len(sentences), len(sentences)])
 for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sentence_similarity_martix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]
 
 sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
 scores = nx.pagerank(sentence_similarity_graph)
 ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(clean)), reverse=True)
 model_out=""
 for i in range(5):
     model_out+=dict[ranked_sentence[i][1]]
     #print(dict[ranked_sentence[i][1]])
 from rouge import Rouge
 print("number of words in the model summary",len(model_out))
 rouge = Rouge()
 scores = rouge.get_scores(model_out, reference)
 scores_rouge1 = [score['rouge-1']['r'] for score in scores]
 scores_rouge2 = [score['rouge-2']['r'] for score in scores]
 scores_rougeL = [score['rouge-l']['r'] for score in scores]                 
 recall1.append(scores_rouge1[0])
 recall2.append(scores_rouge2[0])
 recallL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['p'] for score in scores]
 scores_rouge2 = [score['rouge-2']['p'] for score in scores]
 scores_rougeL = [score['rouge-l']['p'] for score in scores]                 
 precision1.append(scores_rouge1[0])
 precision2.append(scores_rouge2[0])
 precisionL.append(scores_rougeL[0])
 scores_rouge1 = [score['rouge-1']['f'] for score in scores]
 scores_rouge2 = [score['rouge-2']['f'] for score in scores]
 scores_rougeL = [score['rouge-l']['f'] for score in scores]                 
 f1_score1.append(scores_rouge1[0])
 f1_score2.append(scores_rouge2[0])
 f1_scoreL.append(scores_rougeL[0])
 k=k+1

print("the average f1_score for rouge-1 is",math.fsum(f1_score1)/len(f1_score1))
print("the average precision for rouge-1 is",math.fsum(precision1)/len(precision1))
print("the average recall for rouge-1 is",math.fsum(recall1)/len(recall1))
print("the average f1_score for rouge-2 is",math.fsum(f1_score2)/len(f1_score2))
print("the average precision for rouge-2 is",math.fsum(precision2)/len(precision2))
print("the average recall for rouge-2 is",math.fsum(recall2)/len(recall2))
print("the average f1_score for rouge-L is",math.fsum(f1_scoreL)/len(f1_scoreL))
print("the average precision for rouge-L is",math.fsum(precisionL)/len(precisionL))
print("the average recall for rouge-L is",math.fsum(recallL)/len(recallL))
 
 

 


 
 
 
 



['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt', '64.txt', '65.txt', '66.txt', '67.txt', '72.txt', '73.txt', '92.txt', '93.txt']
['102.txt', '104.txt', '105.txt', '106.txt', '108.txt', '109.txt', '110.txt', '111.txt', '118.txt', '128.txt', '129.txt', '130.txt', '131.txt', '132.txt', '148.txt', '149.txt', '150.txt', '151.txt', '152.txt', '153.txt', '155.txt', '156.txt', '159.txt', '160.txt', '161.txt', '163.txt', '168.txt', '169.txt', '17.txt', '170.txt', '173.txt', '174.txt', '181.txt', '19.txt', '205.txt', '206.txt', '214.txt', '225.txt', '227.txt', '228.txt', '229.txt', '230.txt'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:126: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


number of words in the model summary 6823
104.txt
file read successfully
104.txt
file read successfully
number of words in reference summary 4127
number of words in text file 260640
number of sentences in text file 1261


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:126: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


number of words in the model summary 3283
105.txt
file read successfully
105.txt
file read successfully
number of words in reference summary 4498
number of words in text file 264338
number of sentences in text file 1177


...

number of words in the model summary 5837
118.txt
file read successfully
118.txt
file read successfully
number of words in reference summary 847
number of words in text file 189828
number of sentences in text file 886


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:126: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


number of words in the model summary 9449
128.txt
file read successfully
128.txt
file read successfully
number of words in reference summary 2959
number of words in text file 139845
number of sentences in text file 608


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:126: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


number of words in the model summary 3963
the average f1_score for rouge-1 is 0.28985594188232283
the average precision for rouge-1 is 0.29258874317650096
the average recall for rouge-1 is 0.33048813994336784
the average f1_score for rouge-2 is 0.07129852096644654
the average precision for rouge-2 is 0.07131756504431473
the average recall for rouge-2 is 0.08329877025237022
the average f1_score for rouge-L is 0.20325078663677063
the average precision for rouge-L is 0.20027833396741088
the average recall for rouge-L is 0.22641012458502777


In [ ]:
print(f1_score1)
print(f1_score2)
print(f1_scoreL)

[0.3245663072699084, 0.29623567443383847, 0.3420329620363728, 0.31786394925916484, 0.25554483597700894, 0.37508845945259095, 0.28548123481128035, 0.317220538837497, 0.07535515598629772, 0.30917030075926855]
[0.07619047120524164, 0.05409835587589401, 0.07290233338030684, 0.09166135722134303, 0.07528957037958339, 0.12898652956548268, 0.04738561592209513, 0.08770160793413873, 0.02102659094544716, 0.05774277723493267]
[0.20247468566762125, 0.23294508656232968, 0.22773722128300933, 0.20769230270821842, 0.18259022857485782, 0.29729729235253277, 0.17610062393259773, 0.2321839030611971, 0.09315865829526945, 0.1803278639300727]
